In [4]:
import numpy as np
import random



class HexEnv(object):
    kEmpty = 0
    kHorizontal = 1
    kVertical = 2
    def __init__(self, board_size):
        self.board_size = board_size
        self.reset()

    def winner(self):
        return HexEnv.kEmpty

    def to_move(self):
        return self.to_move
    
    @staticmethod
    def make_move(board, action, player):
        # TODO implement
        coords = HexEnv.action_to_coordinate(board, action)
        
        board[2, coords[0], coords[1]] = 0
        board[player, coords[0], coords[1]] = 1
    
    def reset(self):
        self.state = np.zeros((3, self.board_size, self.board_size))
        self.state[2, :, :] = 1.0
        self.to_play = HexEnv.kHorizontal
        self.done = False

        # Let the opponent play if it's not the agent's turn
        if self.player_color != self.to_play:
            a = self.opponent_policy(self.state)
            HexEnv.make_move(self.state, a, HexEnv.kHorizontal)
            self.to_play = HexEnv.kVertical
        return self.state
    

IndentationError: expected an indented block (<ipython-input-4-70b1b7983851>, line 3)

In [ ]:
class Learner(object):
    def __init__(self):
        # TODO set up network
        self.inputs = tf.placeholder(shape=[1,16],dtype=tf.float32)
        W = tf.Variable(tf.random_uniform([16,4],0,0.01))
        self.prob_output_layer = tf.matmul(inputs1,W)
        #self.max_action_layer = tf.argmax(prob_output_layer,1)
       
    @staticmethod
    def rargmax(vector):
        """ Argmax that chooses randomly among eligible maximum indices. """
        m = np.amax(vector)
        indices = np.nonzero(vector == m)[0]
        return random.choice(indices)
        
    def evaluate_prob_dist(self, sess, state):
        prob_output = sess.run([self.prob_output_layer],feed_dict={self.inputs:state})
        return prob_output
    
    def exploration_policy(self, sess, state):
        played = np.logical_or(state[HexEnv.kVertical], state[HexEnv.kHorizontal]).flatten()
        state = np.asarray(state, dtype=theano.config.floatX)
        prob_dist = self.evaluate_prob_dist(sess, state)

        #epsilon greedy
        if np.random.rand()<0.1:
            action = np.random.choice(np.where(played==0)[0])
            return action, prob_dist

        values = np.copy(prob_dist)
        #never select played values
        values[played]=-2
        action = Learner.rargmax(values)
        return action, prob_dist
    
    
    def learn(self, sess, state, batch_size):
        #Do nothing if we don't yet have enough entries in memory for a full batch
        if(self.mem.size < batch_size):
            return
        
        # TODO implement batch sampling
        states1, actions, states2, terminals = self.mem.sample_batch(batch_size)

        # 
        Pw2= self._evaluate_Pws(states2)
        #add a cap on the lowest possible value of losing probability
        Pl =  np.maximum(1-Pw2,0.00001)
        joint = np.min(Pl, axis=1)
        #Update networks
        Pw_targets = np.zeros(terminals.size).astype(theano.config.floatX)
        Pw_targets[terminals==0] = joint[terminals==0]
        Pw_targets[terminals==1] = 1
        return self._update(states1, actions, Pw_targets)[0]

In [2]:
import tensorflow as tf

IMAGE_SIZE = 11

# We'll bundle groups of examples during training for efficiency.
# This defines the size of the batch.
BATCH_SIZE = 1
# We have only one channel in our grayscale images.
NUM_CHANNELS = 3
# The random seed that defines initialization.
SEED = 42

# This is where training samples and labels are fed to the graph.
# These placeholder nodes will be fed a batch of training data at each
# training step, which we'll write once we define the graph structure.
train_data_node = tf.placeholder(
  tf.float32,
  shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
train_labels_node = tf.placeholder(tf.float32,
                                   shape=(BATCH_SIZE, NUM_LABELS))

# For the validation and test data, we'll just hold the entire dataset in
# one constant node.
validation_data_node = tf.constant(validation_data)
test_data_node = tf.constant(test_data)

# The variables below hold all the trainable weights. For each, the
# parameter defines how the variables will be initialized.
conv1_weights = tf.Variable(
  tf.truncated_normal([5, 5, NUM_CHANNELS, 32],  # 5x5 filter, depth 32.
                      stddev=0.1,
                      seed=SEED))
conv1_biases = tf.Variable(tf.zeros([32]))
conv2_weights = tf.Variable(
  tf.truncated_normal([5, 5, 32, 64],
                      stddev=0.1,
                      seed=SEED))
conv2_biases = tf.Variable(tf.constant(0.1, shape=[64]))
fc1_weights = tf.Variable(  # fully connected, depth 512.
  tf.truncated_normal([IMAGE_SIZE // 4 * IMAGE_SIZE // 4 * 64, 512],
                      stddev=0.1,
                      seed=SEED))
fc1_biases = tf.Variable(tf.constant(0.1, shape=[512]))
fc2_weights = tf.Variable(
  tf.truncated_normal([512, NUM_LABELS],
                      stddev=0.1,
                      seed=SEED))
fc2_biases = tf.Variable(tf.constant(0.1, shape=[NUM_LABELS]))

print('Done')

result:  [ 3.  3.  3.  3.]


In [5]:
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt

#These lines establish the feed-forward part of the network used to choose actions
inputs1 = tf.placeholder(shape=[1,16],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4],0,0.01))
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)

#Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
nextQ = tf.placeholder(shape=[1,4],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

init = tf.initialize_all_variables()

# Set learning parameters
y = .99
e = 0.1
num_episodes = 2000
BOARD_SIZE = 11

#create lists to contain total rewards and steps per episode
game_lengths = []

with tf.Session() as sess:
    sess.run(init)
    env = HexEnv(BOARD_SIZE)
    agent = Learner()
    
    for i in range(num_episodes):
        #Reset environment and get first new observation
        game_state = env.reset()
        game_length = 0
        to_move = HexEnv.kHorizontal
        
        while game_state.Winner() != HexEnv.kEmpty:
            game_length += 1
            #Choose an action by greedily (with e chance of random action) from the Q-network
            action, prob_dist = agent.exploration_policy(sess, game_state)
                
            #Get new state and reward from environment
            next_game_state = env.make_move(action, game_state, to_move)
            terminal = 0
            if game_state.winner() != HexEnv.kEmpty:
                terminal = 1
                
            # TODO flip board
            agent.update_memory(game_state, action, next_game_state, terminal)
            prob_dist_costs = agent.learn(sess, batch_size)

            
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = reward + y*maxQ1
            
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:np.identity(16)[game_state:game_state+1],nextQ:targetQ})
            game_state = next_game_state
                        if(costs is not None):
                Pw_cost, Qsigma_cost = costs
            else:
                Pw_cost = 0
                Qsigma_cost = 0

            #update running sums for this episode
            num_step += 1
            Pw_var_sum += np.mean(((1-Pw)*Pw)[np.logical_not(played)])
            Qsigma_sum += np.mean(Qsigma[np.logical_not(played)])
            Qsigma_cost_sum += Qsigma_cost
            Pw_cost_sum += Pw_cost
            
        game_lengths.append(game_length)

print "Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%"

IndentationError: expected an indented block (<ipython-input-5-79bcae2670e7>, line 52)